## In this notebook:

* on feature set 4

### Imports

In [9]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [10]:
#################################################### 
df = pd.read_pickle("./pickled/chr2-4_featureset4.pkl")
df = df.replace({'-': 0})

# problematic isn't first:

#print("LENGTH",len(df.columns))
col = df.pop("Problematic")
df[col.name] = col

# Split our data into three separate sets for training purposes
train, test = train_test_split(df, shuffle=False)
train, valid = train_test_split(train, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                       aapos codonpos  Ancestral_allele  Polyphen2_HVAR_pred  \
chr pos       ref alt                                                          
3   101244651 A   T      560        3                 2                    0   
    101244766 G   C      522        2                 2                    0   
    101269574 C   A       -1        0                 2                    0   
    101275694 G   C      212        2                 2                    0   
    101291499 A   C      171        3                 2                    0   

                          GenoCanyon_score HUVEC_fitCons_score  \
chr pos       ref alt                                            
3   101244651 A   T      0.995787957382137            0.567892   
    101244766 G   C      0.997773309808223            0.567892   
    101269574 C   A      0.998404006175971            0.057018   
    101275694 G   C    1.97468820369911E-4            0.620846   
    101291499 A   C      0.985239277427167 

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [11]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(valid)  
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [12]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(14, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 14))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [ ]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [0/10365 (0%)]	Loss: 0.710937
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [59/10365 (1%)]	Loss: 0.574099
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [118/10365 (1%)]	Loss: 0.573192
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [177/10365 (2%)]	Loss: 0.566553
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [236/10365 (2%)]	Loss: 0.603100
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [295/10365 (3%)]	Loss: 0.618004
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [354/10365 (3%)]	Loss: 0.599571
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [413/10365 (4%)]	Loss: 0.588506
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [472/10365 (5%)]	Loss: 0.583171
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [531/10365 (5%)]	Loss: 0.657578
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [590/10365 (6%)]	Loss: 0.480891
torch.Size([59, 14])
tor

 Train Epoch: 1/20 [5369/10365 (52%)]	Loss: 0.551521
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5428/10365 (52%)]	Loss: 0.618371
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5487/10365 (53%)]	Loss: 0.551497
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5546/10365 (53%)]	Loss: 0.585332
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5605/10365 (54%)]	Loss: 0.517808
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5664/10365 (55%)]	Loss: 0.551874
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5723/10365 (55%)]	Loss: 0.601537
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5782/10365 (56%)]	Loss: 0.551093
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5841/10365 (56%)]	Loss: 0.720040
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5900/10365 (57%)]	Loss: 0.550268
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 1/20 [5959/10365 (57%)]	Loss: 0.516301
torch.Size([59, 14])
torch.Size([59])

 Train Epoch: 2/20 [0/10365 (0%)]	Loss: 0.740367
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [59/10365 (1%)]	Loss: 0.657007
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [118/10365 (1%)]	Loss: 0.603578
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [177/10365 (2%)]	Loss: 0.654990
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [236/10365 (2%)]	Loss: 0.615631
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [295/10365 (3%)]	Loss: 0.548886
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [354/10365 (3%)]	Loss: 0.633508
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [413/10365 (4%)]	Loss: 0.500247
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [472/10365 (5%)]	Loss: 0.527430
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [531/10365 (5%)]	Loss: 0.563478
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [590/10365 (6%)]	Loss: 0.546399
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [649/

 Train Epoch: 2/20 [5369/10365 (52%)]	Loss: 0.550564
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5428/10365 (52%)]	Loss: 0.618347
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5487/10365 (53%)]	Loss: 0.533610
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5546/10365 (53%)]	Loss: 0.635325
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5605/10365 (54%)]	Loss: 0.634227
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5664/10365 (55%)]	Loss: 0.618434
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5723/10365 (55%)]	Loss: 0.550561
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5782/10365 (56%)]	Loss: 0.618345
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5841/10365 (56%)]	Loss: 0.617951
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5900/10365 (57%)]	Loss: 0.618299
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 2/20 [5959/10365 (57%)]	Loss: 0.669193
torch.Size([59, 14])
torch.Size([59])

 Train Epoch: 3/20 [0/10365 (0%)]	Loss: 0.590614
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [59/10365 (1%)]	Loss: 0.653638
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [118/10365 (1%)]	Loss: 0.569306
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [177/10365 (2%)]	Loss: 0.531706
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [236/10365 (2%)]	Loss: 0.558552
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [295/10365 (3%)]	Loss: 0.662453
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [354/10365 (3%)]	Loss: 0.550962
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [413/10365 (4%)]	Loss: 0.549297
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [472/10365 (5%)]	Loss: 0.608432
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [531/10365 (5%)]	Loss: 0.517489
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [590/10365 (6%)]	Loss: 0.614377
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 3/20 [649/

...

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([0., 0., 0.,  ..., 1., 1., 0.])
 Train Epoch: 14/20 [10365/10365 (99%)]	Loss: 0.538262	 Validation Accuracy: 72.22%
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [0/10365 (0%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [59/10365 (1%)]	Loss: 0.567546
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [118/10365 (1%)]	Loss: 0.686143
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [177/10365 (2%)]	Loss: 0.567499
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [236/10365 (2%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [295/10365 (3%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [354/10365 (3%)]	Loss: 0.567505
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [413/10365 (4%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [472/10365 (5%)]	Loss: 0.601397
torch.Siz

 Train Epoch: 15/20 [5192/10365 (50%)]	Loss: 0.679976
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5251/10365 (51%)]	Loss: 0.704253
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5310/10365 (51%)]	Loss: 0.594933
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5369/10365 (52%)]	Loss: 0.584254
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5428/10365 (52%)]	Loss: 0.661417
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5487/10365 (53%)]	Loss: 0.516686
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5546/10365 (53%)]	Loss: 0.601357
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5605/10365 (54%)]	Loss: 0.482753
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5664/10365 (55%)]	Loss: 0.516651
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5723/10365 (55%)]	Loss: 0.618346
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 15/20 [5782/10365 (56%)]	Loss: 0.618346
torch.Size([59, 14])
torch

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([0., 0., 0.,  ..., 1., 1., 0.])
 Train Epoch: 15/20 [10365/10365 (99%)]	Loss: 0.513262	 Validation Accuracy: 72.22%
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [0/10365 (0%)]	Loss: 0.516651
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [59/10365 (1%)]	Loss: 0.703092
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [118/10365 (1%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [177/10365 (2%)]	Loss: 0.618346
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [236/10365 (2%)]	Loss: 0.703092
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [295/10365 (3%)]	Loss: 0.567499
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [354/10365 (3%)]	Loss: 0.703092
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [413/10365 (4%)]	Loss: 0.686143
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [472/10365 (5%)]	Loss: 0.635296
torch.Siz

 Train Epoch: 16/20 [5192/10365 (50%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5251/10365 (51%)]	Loss: 0.499702
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5310/10365 (51%)]	Loss: 0.669194
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5369/10365 (52%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5428/10365 (52%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5487/10365 (53%)]	Loss: 0.533600
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5546/10365 (53%)]	Loss: 0.720041
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5605/10365 (54%)]	Loss: 0.533600
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5664/10365 (55%)]	Loss: 0.499702
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5723/10365 (55%)]	Loss: 0.618346
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 16/20 [5782/10365 (56%)]	Loss: 0.567499
torch.Size([59, 14])
torch

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([0., 0., 0.,  ..., 1., 1., 0.])
 Train Epoch: 16/20 [10365/10365 (99%)]	Loss: 0.588261	 Validation Accuracy: 72.22%
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [0/10365 (0%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [59/10365 (1%)]	Loss: 0.618346
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [118/10365 (1%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [177/10365 (2%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [236/10365 (2%)]	Loss: 0.550550
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [295/10365 (3%)]	Loss: 0.533600
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [354/10365 (3%)]	Loss: 0.601397
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [413/10365 (4%)]	Loss: 0.584448
torch.Size([59, 14])
torch.Size([59])
 Train Epoch: 17/20 [472/10365 (5%)]	Loss: 0.720042
torch.Siz

### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [ ]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]


y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))

print(len(pred))
print(len(y_labels))

### Metrics
* Use sklearn builtins to calculate different metrics

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_labels, pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_labels, pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_labels, pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_labels, pred, average=None)
print("Precision TP/(TP+FP):",precision)


# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

In [ ]:
# Recalculating accuracy: TP + TN / (all exmaples)
print("accuracy", 2362/(1080+2362))
1080+2362

#Test shape, train shape
#(10325, 9) (3442, 9)